# Práctica Regresión Lineal, Múltiple y Logística

Autor: Adrián Arroyo Calle




Usando el conocido conjunto de datos IRIS, realice una clasificación mediante regresión lineal múltiple. Para ello, se destinará ⅔ de los datos escogidos aleatoriamente de manera estratificada para aprendizaje y, el resto, para verificación.

In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.model_selection import StratifiedKFold

Cargamos el dataset IRIS

In [17]:
iris = load_iris(return_X_y=False)

Normalizamos la entrada al rango `[0,1]`

In [49]:
scaler = MinMaxScaler()
scaled_iris_data = scaler.fit_transform(iris.data)

Binarizamos las clases de salida, para usar varios regresores lineales

In [50]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
bin_iris_target = lb.fit_transform(iris.target)

Hacemos la separación Holdout 33%, estratificada

In [55]:
x_train,x_test,y_train,y_test = train_test_split(iris.data,bin_iris_target,test_size=0.33,stratify=bin_iris_target)

In [71]:
y_test.shape[1]

3

In [76]:
y_predict = np.zeros(y_test.shape, dtype=float)
for i in range(y_test.shape[1]):
    reg = LinearRegression()
    reg.fit(x_train,y_train[:,i])
    y_predict[:,i] = reg.predict(x_test)

Ahora con los datos de prueba analizamos con qué regresor se obtiene más probabilidad

In [80]:
score = np.sum(np.argmax(y_predict,axis=1) == np.argmax(y_test, axis=1))/y_test.shape[0]
print("Tasa de acierto %f" % score)

Tasa de acierto 0.860000


Matriz de confusión

In [91]:
s = accuracy_score(np.argmax(y_test,axis=1), np.argmax(y_predict, axis=1))
m = confusion_matrix(np.argmax(y_test,axis=1), np.argmax(y_predict, axis=1))
m

array([[16,  0,  0],
       [ 0, 12,  5],
       [ 0,  2, 15]])